In [1]:
#from textblob import TextBlob
#from attributegetter import *
from generatengrams import ngrammatch
from Contexts import *
import json
from Intents import *
import random
import os
import re
import pandas as pd
from spell import correction, words

ModuleNotFoundError: ignored

In [3]:
!pip install generatengrams

  Could not find a version that satisfies the requirement generatengrams (from versions: )
No matching distribution found for generatengrams


In [0]:
def perform_action_restaurant_booking(attributes):
    df=pd.read_csv("restaurantDB.csv")

    restaurant = df[(df.location== attributes['location']) & 
                    (df.cusine==attributes['cusine'])& 
                    (df.cost==attributes['cost'])].values[0][3]
    return ("You have made a booking at " + restaurant), IntentComplete()

def check_actions(current_intent, attributes, context):
    '''This function performs the action for the intent
    as mentioned in the intent config file'''
    '''Performs actions pertaining to current intent '''
    
    context = IntentComplete()
    #print(" intent " +str(current_intent.action))
    #print (str(attributes))
   
    #try:
    if (current_intent.action == "SearchRestaurant") :
            return perform_action_restaurant_booking(attributes)
    if (current_intent.action == "BookCab") :
            return perform_action_cab_booking(attributes)
   # except:
   #     print("ignore")   
    return 'action: ' + current_intent.action, context

def readCabCSV(destination,people,lugguage): 
        result = "Cabs are not available for your location and preferences.\n Please try changing your preferences."
        if people:
            people=people.lower()
        if(people in ["one people","two people","three people" ,"four people"]):
             maxpeople= 4 
        elif(people in ["five people","six people","seven people"]):
              maxpeople= 7
        elif(int(people) <=3):
             maxpeople =3
        elif(int(people) <=4):
             maxpeople =4
        elif(int(people) <=7):
             maxpeople =7
        else:
             maxpeople = int(people)
    
        if lugguage:
            lugguage=lugguage.lower() 
        if(lugguage in ["one bag","two bags"]):
            maxlugguage= 2 ;
        elif(lugguage in ["three bags"]):     
             maxlugguage=3;
        elif(lugguage in ["four bags","five bags","six bags","seven bags","seven bags"]):
             maxlugguage= 8 ;     
        elif(int(lugguage) <=2):
             maxlugguage =2
        elif(int(lugguage) <=3):
             maxlugguage =3
        elif(int(lugguage) <=8):
             maxlugguage =8
        else:
            maxlugguage=int(lugguage)
            
        print("maxlugguage=",maxlugguage)
        print("maxpeople=",maxpeople)
        data = pd.read_csv("db/cabsavailable.csv")

        #print("Initial data:\n",data)
        if(len(data.index) != 0):
            data = data.loc[data['location'] == ('madhapur')]
            print("After location filter:\n",data)
        else:
            return result
        if(len(data.index) != 0):
            data = data[data['maxnumber'] == maxpeople]
            print("After people filter:\n",data)
        else:
            return result
        if(len(data.index) != 0):
            data = data[data['maxluggage'] == maxlugguage]
            print("After lugguage filter:\n",data)
        else:
            return result
        result = ""
        if(len(data.index) != 0):
            result = "Cab booked :" + data.values[0][1]+" - "+data.values[0][4]    
        return result
def perform_action_cab_booking(attributes):
    result = readCabCSV(attributes['destination'],attributes['people'],attributes['luggage'])
    return result, IntentComplete()


def check_required_params(current_intent, attributes, context):
    '''Collects attributes pertaining to the current intent'''
    
    for para in current_intent.params:
        if para.required:
            if para.name not in attributes:
                #Example of where the context is born, implemented in Contexts.py
                if para.name=='RegNo':
                    context = GetRegNo()
                if para.name=='people':
                    context = GetPeopleNo()
                if para.name=='luggage':
                    context = GetLuggageNo()
                #returning a random prompt frmo available choices.
                return random.choice(para.prompts), context

    return None, context


def input_processor(user_input, context, attributes, intent):
    '''Spellcheck and entity extraction functions go here'''
    
    list_words = words(user_input)
    corrected_user_input = ""
    for word in list_words:
      corrected_user_input+=correction(word)+" "
    
    #print("Preprocessed check")
    print(corrected_user_input)
    #uinput = TextBlob(user_input).correct().string
    
    #update the attributes, abstract over the entities in user input
    attributes, cleaned_input = getattributes(corrected_user_input, context, attributes)
    
    return attributes, cleaned_input

def loadIntent(path, intent):
    with open(path) as fil:
        dat = json.load(fil)
        intent = dat[intent]
        return Intent(intent['intentname'],intent['Parameters'], intent['actions'])

def intentIdentifier(clean_input, context,current_intent):
    clean_input = clean_input.lower()
    
    #Scoring Algorithm, can be changed.
    scores = ngrammatch(clean_input)
    
    #choosing here the intent with the highest score
    scores = sorted_by_second = sorted(scores, key=lambda tup: tup[1])
    print("clean input: ", clean_input)
    print("scores: ", scores)
    

    if(current_intent==None):
        if(clean_input=="search"):
            return loadIntent('params/newparams.cfg', 'SearchRestaurant')
        if(clean_input=='book'):
            return loadIntent('params/newparams.cfg','BookCab')
        else:
            return loadIntent('params/newparams.cfg',scores[-1][0])
    else:
        #If current intent is not none, stick with the ongoing intent
        return current_intent

def getattributes(uinput,context,attributes):
    '''This function marks the entities in user input, and updates
    the attributes dictionary'''
    #Can use context to context specific attribute fetching
    if context.name.startswith('IntentComplete'):
        return attributes, uinput
    else:
        #Code can be optimised here, loading the same files each time suboptimal 
        files = os.listdir('./entities/')
        entities = {}
        for fil in files:
            lines = open('./entities/'+fil).readlines()
            for i, line in enumerate(lines):
                lines[i] = line[:-1]
            entities[fil[:-4]] = '|'.join(lines)
        #print("Entities: ", entities)
        #print("Uinput 1", uinput)

        #Extract entity and update it in attributes dict
        for entity in entities:
            for i in entities[entity].split('|'):
                if i.lower() in uinput.lower():
                    attributes[entity] = i
        for entity in entities:
                print("Entity 1", entities[entity])
                uinput = re.sub(entities[entity],r'$'+entity,uinput,flags=re.IGNORECASE)
        print("Uinput 2", uinput)

        #Example of where the context is being used to do conditional branching.
        if context.name=='GetRegNo' and context.active:
            print(attributes)
            match = re.search('[0-9]+', uinput)
            if match:
                uinput = re.sub('[0-9]+', '$regno', uinput)
                attributes['RegNo'] = match.group()
                context.active = False
                
        print("Context is ", context.name)
        if context.name=='BookCab_people' and context.active:
            print(attributes)
            match = re.search('[0-9]+', uinput)
            if match:
                uinput = re.sub('[0-9]+', '$people', uinput)
                attributes['people'] = match.group()
                context.active = False
                
        if context.name=='BookCab_lugg' and context.active:
            print(attributes)
            match = re.search('[0-9]+', uinput)
            if match:
                uinput = re.sub('[0-9]+', '$luggage', uinput)
                attributes['luggage'] = match.group()
                context.active = False
        return attributes, uinput

In [5]:
class Session:
    def __init__(self, attributes=None, active_contexts=[FirstGreeting(), IntentComplete() ]):
        
        '''Initialise a default session'''
        
        #Active contexts not used yet, can use it to have multiple contexts
        self.active_contexts = active_contexts
        
        #Contexts are flags which control dialogue flow, see Contexts.py        
        self.context = FirstGreeting()
        
        #Intent tracks the current state of dialogue
        #self.current_intent = First_Greeting()
        self.current_intent = None
        
        #attributes hold the information collected over the conversation
        self.attributes = {}
        
    def update_contexts(self):
        '''Not used yet, but is intended to maintain active contexts'''
        for context in self.active_contexts:
            if context.active:
                context.decrease_lifespan()

    def reply(self, user_input):
        '''Generate response to user input'''
        
        self.attributes, clean_input = input_processor(user_input, self.context, self.attributes, self.current_intent)
        
        self.current_intent = intentIdentifier(clean_input, self.context, self.current_intent)
        
        prompt, self.context = check_required_params(self.current_intent, self.attributes, self.context)

        #prompt being None means all parameters satisfied, perform the intent action
        if prompt is None:
            if self.context.name!='IntentComplete':
                prompt, self.context = check_actions(self.current_intent, self.attributes, self.context)

                
                
        #Resets the state after the Intent is complete
        if self.context.name=='IntentComplete':
            self.attributes = {}
            self.context = FirstGreeting()
            self.current_intent = None
        
        return prompt

NameError: ignored

In [6]:
session = Session()

print ('BOT: Hi! How may I assist you?')

while True:
           inp = input('User: ')
           if inp=='exit':
                break
           elif inp=='stop':
               break
           elif inp=='end':
               break
           elif inp=='restart':
               del session
               session=Session()
               print ('BOT: Hi! How may I assist you?')
           else:
               print ('BOT:', session.reply(inp))

NameError: ignored